In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import sqlalchemy
from geopy.point import Point

In [2]:
from math import sin, cos, sqrt, fabs, atan2
from math import pi

#---------------------------------------------------#
# 因地图坐标偏差加入辅助的转换函数
# 作者：元凿坊工作室
# 链接：https://zhuanlan.zhihu.com/p/107253611
#--------------------------------------------------#

# define ellipsoid
a = 6378245
f = 1 / 298.3
b = a * (1 - f)
ee = 1 - (b * b) / (a * a)

# check if the point in china
def outOfChina(lng, lat):
    return not (72.004 <= lng <= 137.8347 and 0.8293 <= lat <= 55.8271)


def geohey_transformLat(x, y):
    ret = -100.0 + 2.0 * x + 3.0 * y + 0.2 * y * y + 0.1 * x * y + 0.2 * sqrt(fabs(x))
    ret = ret + (20.0 * sin(6.0 * x * pi) + 20.0 * sin(2.0 * x * pi)) * 2.0 / 3.0
    ret = ret + (20.0 * sin(y * pi) + 40.0 * sin(y / 3.0 * pi)) * 2.0 / 3.0
    ret = ret + (160.0 * sin(y / 12.0 * pi) + 320.0 * sin(y * pi / 30.0)) * 2.0 / 3.0
    return ret


def geohey_transformLon(x, y):
    ret = 300.0 + x + 2.0 * y + 0.1 * x * x +  0.1 * x * y + 0.1 * sqrt(fabs(x))
    ret = ret + (20.0 * sin(6.0 * x * pi) + 20.0 * sin(2.0 * x * pi)) * 2.0 / 3.0
    ret = ret + (20.0 * sin(x * pi) + 40.0 * sin(x / 3.0 * pi)) * 2.0 / 3.0
    ret = ret + (150.0 * sin(x / 12.0 * pi) + 300.0 * sin(x * pi / 30.0)) * 2.0 / 3.0
    return ret


def wgs2gcj(wgsLon, wgsLat):
    if outOfChina(wgsLon, wgsLat):
        return wgsLon, wgsLat
    dLat = geohey_transformLat(wgsLon - 105.0, wgsLat - 35.0)
    dLon = geohey_transformLon(wgsLon - 105.0, wgsLat - 35.0)
    radLat = wgsLat / 180.0 * pi
    magic = sin(radLat)
    magic = 1 - ee * magic * magic
    sqrtMagic = sqrt(magic)
    dLat = (dLat * 180.0) / ((a * (1 - ee)) / (magic * sqrtMagic) * pi)
    dLon = (dLon * 180.0) / (a / sqrtMagic * cos(radLat) * pi)
    gcjLat = wgsLat + dLat
    gcjLon = wgsLon + dLon
    return (gcjLon, gcjLat)


def gcj2wgs(gcjLon, gcjLat):
    g0 = (gcjLon, gcjLat)
    w0 = g0
    g1 = wgs2gcj(w0[0], w0[1])
    # w1 = w0 - (g1 - g0)
    w1 = tuple([x[0]-(x[1]-x[2]) for x in zip(w0,g1,g0)])  
    # delta = w1 - w0
    delta = tuple([x[0] - x[1] for x in zip(w1, w0)])
    while (abs(delta[0]) >= 1e-6 or abs(delta[1]) >= 1e-6):
        w0 = w1
        g1 = wgs2gcj(w0[0], w0[1])
        # w1 = w0 - (g1 - g0)
        w1 = tuple([x[0]-(x[1]-x[2]) for x in zip(w0,g1,g0)])
        # delta = w1 - w0
        delta = tuple([x[0] - x[1] for x in zip(w1, w0)])
    return w1

In [3]:
Engine = sqlalchemy.create_engine('sqlite:///lianjia.db')

HouseExtractionSQL = '''
                     SELECT ID, District, Community, RentType, Area, Price, 
                     Price/Area AS UnitPrice, HouseFloor, BuldFloor, ElevatorFlag
                     FROM `guangzhou-detail`
                     '''
HouseData = pd.read_sql_query(HouseExtractionSQL, Engine)

In [4]:
# Simple data wrangling of HouseData
HouseID = HouseData['ID']
HouseData.drop(['ID'], axis=1, inplace=True)

# Convert descriptive house floor into abbrevation
HouseData['HouseFloor'] = HouseData['HouseFloor'].replace({'低楼层': 'L', '中楼层': 'M', '高楼层': 'H', '地下室': 'B'})
# Convert rent type into English
HouseData['RentType'] = HouseData['RentType'].replace({'整租':'Whole', '合租':'Shared'})
# Convert elevator status into 1/0 form
HouseData['ElevatorFlag'] = HouseData['ElevatorFlag'].replace({'有':1, '无':0})
HouseData.head()

,District,Community,RentType,Area,Price,UnitPrice,HouseFloor,BuldFloor,ElevatorFlag
0,海珠,洪德路,Whole,68,5000,73,L,4,0
1,海珠,宏宇广场罗马假日,Whole,120,6500,54,H,22,1
2,南沙,香江国际金融中心,Whole,39,1800,46,H,22,1
3,南沙,越秀滨海隽城,Whole,89,2600,29,L,18,1
4,海珠,汇美南苑,Whole,92,6500,70,M,9,1


In [5]:
CommunityExtractionSQL = '''
                        SELECT * FROM `guangzhou-community`
                        WHERE (Longitude IS NOT NULL) AND (Latitude IS NOT NULL)
                        '''
CommunityData = pd.read_sql_query(CommunityExtractionSQL, Engine)

# Convert to WGS coordinate
CommunityData['Geometry'] = CommunityData.apply(lambda x: Point(gcj2wgs(x['Longitude'], x['Latitude'])[::-1]), axis=1)
CommunityData.drop(['Longitude', 'Latitude'], axis=1, inplace=True)
CommunityData.rename(columns={'ID': 'CommunityID'}, inplace=True)
CommunityData.head()

,CommunityID,District,Community,Geometry
0,1,荔湾,芳村大道西,"23 6m 45.2885s N, 113 12m 36.7962s E"
1,2,天河,荟雅苑,"23 8m 42.5338s N, 113 19m 18.6335s E"
2,3,番禺,新月明珠花园,"23 1m 7.92681s N, 113 18m 50.7379s E"
3,4,荔湾,周门小区周门西街,"23 7m 56.8195s N, 113 13m 57.8302s E"
4,5,番禺,禺秀园,"22 56m 38.1827s N, 113 21m 13.4838s E"


In [6]:
MetroExtractionSQL = '''
                     SELECT * FROM `guangzhou-metro`
                     WHERE (Longitude IS NOT NULL) AND (Latitude IS NOT NULL)
                     '''
MetroData = pd.read_sql_query(MetroExtractionSQL, Engine)

# Due to the naming convention of Guangzhou Metro, the unconnected part of Line 3 and Line 14 need to be cleaned
MetroData.loc[MetroData['LineName']=='三北线', 'LineCode'] = '3N'
MetroData.loc[MetroData['LineName'].str.contains('知识城'), 'LineCode'] = '14B'
# Interchange station of Line 3 and Line 3N
L3_ex_stat = MetroData[(MetroData['LineCode'] == 3) & (MetroData['StationName'] == '体育西路')].copy() 
L3_ex_stat.replace({'LineCode': {3: '3N'}, 'LineName': {'三号线': '三北线'}}, inplace=True)
MetroData = pd.concat([MetroData, L3_ex_stat], ignore_index=True)

L14_ex_stat = MetroData[(MetroData['LineCode'] == 14) & (MetroData['StationName'] == '新和')].copy()
# Interchange station of Line 14 and Line 14B
L14_ex_stat.replace({'LineCode': {14: '14B'}, 'LineName': {'十四号线': '十四号线(知识城)'}}, inplace=True) 
MetroData = pd.concat([MetroData, L14_ex_stat], ignore_index=True)

MetroData.tail()

,LineCode,LineName,LineColor,StationCode,StationName,Longitude,Latitude
269,GF,广佛线,"rgb(187, 216, 10)",23,石溪,113.285951,23.067937
270,GF,广佛线,"rgb(187, 216, 10)",24,南洲,113.297367,23.064799
271,GF,广佛线,"rgb(187, 216, 10)",25,沥滘,113.319077,23.054898
272,3N,三北线,"rgb(232, 158, 71)",11,体育西路,113.321503,23.131138
273,14B,十四号线(知识城),"rgb(121, 39, 32)",8,新和,113.467060,23.413259


In [7]:
MetroData = MetroData.sort_values(['LineCode', 'StationCode'], ignore_index=True)

# Convert to WGS coordinate
MetroData['Geometry'] = MetroData.apply(lambda x: Point(gcj2wgs(x['Longitude'], x['Latitude'])[::-1]), axis=1)
MetroData.drop(['LineName', 'LineColor', 'Longitude', 'Latitude'], axis=1, inplace=True)
MetroData.tail()

,LineCode,StationCode,StationName,Geometry
269,GF,21,沙园,"23 5m 28.7543s N, 113 15m 18.9411s E"
270,GF,22,燕岗,"23 4m 40.2186s N, 113 15m 59.9769s E"
271,GF,23,石溪,"23 4m 14.2478s N, 113 16m 50.1949s E"
272,GF,24,南洲,"23 4m 2.9038s N, 113 17m 31.235s E"
273,GF,25,沥滘,"23 3m 27.1465s N, 113 18m 49.2519s E"


In [8]:
House = pd.merge(HouseData, CommunityData, on=['District', 'Community'])

# Map the Chinese district name into abbreviation
DistrictMapping = {'天河':'TH', '番禺':'PY', '白云':'BY', '海珠':'HZ', '花都':'HD', '南沙':'NS',
                   '增城':'ZC', '荔湾':'LW', '越秀':'YX', '黄埔':'HP', '从化':'CH'}
House['District'] = House['District'].replace(DistrictMapping)
House = House.drop(['Community'], axis=1)

In [9]:
import re

# Other minor cleaning due to special input
House.loc[House['RentType'].str.startswith('整租'), 'RentType'] = 'Whole'
House['ElevatorFlag'] = House['ElevatorFlag'].astype(str)
House['ElevatorFlag'] = House['ElevatorFlag'].where(lambda x: x.str.match(r'\d'), np.nan)
House.head()

,District,RentType,Area,Price,UnitPrice,HouseFloor,BuldFloor,ElevatorFlag,CommunityID,Geometry
0,HZ,Whole,120,6500,54,H,22,1,1461,"23 6m 16.3102s N, 113 15m 26.6968s E"
1,NS,Whole,39,1800,46,H,22,1,331,"22 47m 48.0202s N, 113 32m 28.4519s E"
2,NS,Whole,39,1800,46,H,22,1,331,"22 47m 48.0202s N, 113 32m 28.4519s E"
3,NS,Whole,40,1700,42,L,22,1,331,"22 47m 48.0202s N, 113 32m 28.4519s E"
4,NS,Whole,89,2600,29,L,18,1,131,"22 47m 35.5088s N, 113 31m 46.0103s E"


In [10]:
# Write to corresponding file
House.to_csv('houses.csv', index=False)
MetroData.to_csv('metro.csv', index=False)